In [78]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [79]:
import pandas as pd

# Load the datasets
filtered_highways_traffic_data = pd.read_csv('filtered highways traffic data.csv')
filtered_data_last_6_years = pd.read_csv('filtered_data_last_6_years.csv')

# Step 1: Ensure "H" column in filtered_data_last_6_years is in the correct format (integer representing hour)
filtered_data_last_6_years['H'] = filtered_data_last_6_years['H'].astype(int)

# Step 2: Extract hour from "TijdFileBegin" and add as column "H" in filtered_highways_traffic_data
filtered_highways_traffic_data['TijdFileEind'] = pd.to_datetime(filtered_highways_traffic_data['TijdFileEind'], format='%H:%M:%S')
filtered_highways_traffic_data['TijdFileBegin'] = pd.to_datetime(filtered_highways_traffic_data['TijdFileBegin'], format='%H:%M:%S')
filtered_highways_traffic_data['H'] = filtered_highways_traffic_data['TijdFileBegin'].dt.hour

# Convert 'DatumFileBegin' to a common format in both dataframes
filtered_highways_traffic_data['DatumFileBegin'] = pd.to_datetime(filtered_highways_traffic_data['DatumFileBegin'], format='%d/%m/%Y')
filtered_data_last_6_years['YYYYMMDD'] = pd.to_datetime(filtered_data_last_6_years['YYYYMMDD'], format='%d/%m/%Y')


# Convert date columns to datetime format if they are not already, specifying dayfirst=True
filtered_highways_traffic_data['DatumFileBegin'] = pd.to_datetime(filtered_highways_traffic_data['DatumFileBegin'], errors='coerce', dayfirst=True)
filtered_highways_traffic_data['DatumFileEind'] = pd.to_datetime(filtered_highways_traffic_data['DatumFileEind'], errors='coerce', dayfirst=True)

# Step 3: Map RouteNum to weather stations
weather_station_mapping = {
    15: 350,
    50: 275,
    2: 370,
    325: 275
}

filtered_highways_traffic_data['STN'] = filtered_highways_traffic_data['RouteNum'].map(weather_station_mapping)

# Step 4: Join the datasets on "DatumFileBegin" and "H"
# Ensure we join the traffic data (right) to the weather data (left)
merged_data = pd.merge(
    filtered_highways_traffic_data,
    filtered_data_last_6_years,
    left_on=['DatumFileBegin', 'H', 'STN'],
    right_on=['YYYYMMDD', 'H', 'STN'],
    how='right'
)

merged_data


C:\Users\botna\AppData\Local\Temp\ipykernel_4292\3980289062.py:5: DtypeWarning: Columns (12,15,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_data_last_6_years = pd.read_csv('filtered_data_last_6_years.csv')


,NLSitNummer,DatumFileBegin,DatumFileEind,TijdFileBegin,TijdFileEind,FileZwaarte,GemLengte,FileDuur,HectometerKop,HectometerStaart,...,VV,N,U,WW,IX,M,R,S,O,Y
0,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,31,8,97,10,7,0,0,0,0,0
1,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,25,8,96,10,7,0,0,0,0,0
2,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,29,8,94,10,7,0,0,0,0,0
3,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,50,8,94,10,7,0,0,0,0,0
4,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,58,8,94,10,7,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378984,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,75,8,73,,5,0,0,0,0,0
378985,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,75,8,79,,5,0,0,0,0,0
378986,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,75,8,73,,5,0,0,0,0,0
378987,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,75,8,77,,5,0,0,0,0,0


In [80]:
# Correct the hours in 'H' column and convert to time type
merged_data['H'] = merged_data['H'].apply(lambda x: f'{x:02d}:00:00' if x != 24 else '00:00:00')
merged_data['H'] = pd.to_datetime(merged_data['H'], format='%H:%M:%S').dt.time


In [81]:
merged_data.isna().sum()

NLSitNummer                155923
DatumFileBegin             155923
DatumFileEind              155923
TijdFileBegin              155923
TijdFileEind               155923
FileZwaarte                155923
GemLengte                  155923
FileDuur                   155923
HectometerKop              155923
HectometerStaart           155923
RouteLet                   155923
RouteNum                   155923
RouteOms                   155923
hectometreringsrichting    155923
KopWegvakVan               155926
KopWegvakNaar              155923
TrajVan                    155923
TrajNaar                   155923
OorzaakGronddetail         155923
OorzaakVerloop             155923
OorzaakCodeVerloop         155923
OorzaakCode                155923
Oorzaak_1                  155923
Oorzaak_2                  155923
Oorzaak_3                  155923
Oorzaak_4                  155926
H                               0
STN                             0
YYYYMMDD                        0
DD            


| Column Name (English)      | Unit           | Explanation                                                |
|----------------------------|----------------|------------------------------------------------------------|
| NLSituationNumber          | -              | Identifier for the traffic situation                       |
| DateFileStart              | YYYY-MM-DD     | Start date of the traffic congestion                       |
| DateFileEnd                | YYYY-MM-DD     | End date of the traffic congestion                         |
| TimeFileStart              | HH:MM:SS       | Start time of the traffic congestion                       |
| TimeFileEnd                | HH:MM:SS       | End time of the traffic congestion                         |
| FileSeverity               | -              | Severity of the traffic congestion                         |
| AvgLength                  | Kilometers     | Average length of the traffic congestion                   |
| FileDuration               | Minutes        | Duration of the traffic congestion                         |
| HectometerHead             | Hectometer     | Head of the hectometer segment                             |
| HectometerTail             | Hectometer     | Tail of the hectometer segment                             |
| RouteLetter                | -              | Letter representing the route                              |
| RouteNumber                | -              | Number representing the route                              |
| RouteDescription           | -              | Description of the route                                   |
| HectometerDirection        | -              | Direction of the hectometer segment (increasing/decreasing)|
| HeadSegmentFrom            | -              | Starting segment of the head                               |
| HeadSegmentTo              | -              | Ending segment of the head                                 |
| TrajectoryFrom             | -              | Starting point of the trajectory                           |
| TrajectoryTo               | -              | Ending point of the trajectory                             |
| CauseGroundDetail          | -              | Detailed cause of the traffic congestion on the ground     |
| CauseProgress              | -              | Progression of the cause of the traffic congestion         |
| CauseCodeProgress          | -              | Code representing the progression of the cause             |
| CauseCode                  | -              | Code representing the cause                                |
| Cause1                     | -              | Primary cause of the traffic congestion                    |
| Cause2                    | -              | Secondary cause of the traffic congestion                  |
| Cause3                     | -              | Tertiary cause of the traffic congestion                   |
| Cause4                     | -              | Quaternary cause of the traffic congestion                 |
| Station                    | -              | Weather station identifier                                 |
| Date                       | YYYYMMDD       | Date of the weather observation                            |
| Hour                       | Hours          | Hour of the weather observation                            |
| WindDirection              | Degrees        | Direction of the wind                                      |
| WindSpeedAvg               | Meters/second  | Average wind speed                                         |
| WindSpeed                  | Meters/second  | Wind speed                                                 |
| WindGust                   | Meters/second  | Gust speed of the wind                                     |
| Temperature                | Degrees Celsius| Air temperature                                            |
| MinTemp10cm                | Degrees Celsius| Minimum temperature at 10 cm above ground                  |
| DewPoint                   | Degrees Celsius| Dew point temperature                                      |
| SunshineDuration           | Hours          | Duration of sunshine                                       |
| GlobalRadiation            | Joules/cm^2    | Global radiation                                           |
| PrecipitationDuration      | Hours          | Duration of precipitation                                  |
| PrecipitationAmount        | Millimeters    | Amount of precipitation                                    |
| AirPressure                | Hectopascals   | Air pressure                                               |
| Visibility                 | Meters         | Visibility distance                                        |
| CloudCover                 | -              | Cloud cover percentage                                     |
| Humidity                   | Percentage     | Relative humidity                                          |
| Weather                    | -              | General weather description                                |
| WeatherCode                | -              | Code representing the weather                              |
| Fog                        | Boolean        | Presence of fog                                            |
| Rain                       | Boolean        | Presence of rain                                           |
| Snow                       | Boolean        | Presence of snow                                           |
| Thunder                    | Boolean        | Presence of thunder                                        |
| IceFormation               | Boolean        | Presence of ice formation                                  |


In [82]:
dtypes_df = pd.DataFrame(merged_data.dtypes, columns=['Data Type'])
dtypes_df

,Data Type
NLSitNummer,float64
DatumFileBegin,datetime64[ns]
DatumFileEind,datetime64[ns]
TijdFileBegin,datetime64[ns]
TijdFileEind,datetime64[ns]
FileZwaarte,object
GemLengte,object
FileDuur,object
HectometerKop,object
HectometerStaart,object


In [83]:
# Define the new column names
column_mapping = {
    'NLSitNummer': 'NLSituationNumber',
    'DatumFileBegin': 'DateFileStart',
    'DatumFileEind': 'DateFileEnd',
    'TijdFileBegin': 'TimeFileStart',
    'TijdFileEind': 'TimeFileEnd',
    'FileZwaarte': 'FileSeverity',
    'GemLengte': 'AvgLength',
    'FileDuur': 'FileDuration',
    'HectometerKop': 'HectometerHead',
    'HectometerStaart': 'HectometerTail',
    'RouteLet': 'RouteLetter',
    'RouteNum': 'RouteNumber',
    'RouteOms': 'RouteDescription',
    'hectometreringsrichting': 'HectometerDirection',
    'KopWegvakVan': 'HeadSegmentFrom',
    'KopWegvakNaar': 'HeadSegmentTo',
    'TrajVan': 'TrajectoryFrom',
    'TrajNaar': 'TrajectoryTo',
    'OorzaakGronddetail': 'CauseGroundDetail',
    'OorzaakVerloop': 'CauseProgress',
    'OorzaakCodeVerloop': 'CauseCodeProgress',
    'OorzaakCode': 'CauseCode',
    'Oorzaak_1': 'Cause1',
    'Oorzaak_2': 'Cause2',
    'Oorzaak_3': 'Cause3',
    'Oorzaak_4': 'Cause4',
    'STN': 'Station',
    'YYYYMMDD': 'Date',
    'H': 'Hour',
    'DD': 'WindDirection',
    'FH': 'WindSpeedAvg',
    'FF': 'WindSpeed',
    'FX': 'WindGust',
    'T': 'Temperature',
    'T10N': 'MinTemp10cm',
    'TD': 'DewPoint',
    'SQ': 'SunshineDuration',
    'Q': 'GlobalRadiation',
    'DR': 'PrecipitationDuration',
    'RH': 'PrecipitationAmount',
    'P': 'AirPressure',
    'VV': 'Visibility',
    'N': 'CloudCover',
    'U': 'Humidity',
    'WW': 'Weather',
    'IX': 'WeatherCode',
    'M': 'Fog',
    'R': 'Rain',
    'S': 'Snow',
    'O': 'Thunder',
    'Y': 'IceFormation'
}

# Rename the columns
merged_data.rename(columns=column_mapping, inplace=True)

merged_data

,NLSituationNumber,DateFileStart,DateFileEnd,TimeFileStart,TimeFileEnd,FileSeverity,AvgLength,FileDuration,HectometerHead,HectometerTail,...,Visibility,CloudCover,Humidity,Weather,WeatherCode,Fog,Rain,Snow,Thunder,IceFormation
0,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,31,8,97,10,7,0,0,0,0,0
1,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,25,8,96,10,7,0,0,0,0,0
2,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,29,8,94,10,7,0,0,0,0,0
3,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,50,8,94,10,7,0,0,0,0,0
4,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,58,8,94,10,7,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378984,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,75,8,73,,5,0,0,0,0,0
378985,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,75,8,79,,5,0,0,0,0,0
378986,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,75,8,73,,5,0,0,0,0,0
378987,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,75,8,77,,5,0,0,0,0,0


In [84]:
merged_data.dtypes

NLSituationNumber               float64
DateFileStart            datetime64[ns]
DateFileEnd              datetime64[ns]
TimeFileStart            datetime64[ns]
TimeFileEnd              datetime64[ns]
FileSeverity                     object
AvgLength                        object
FileDuration                     object
HectometerHead                   object
HectometerTail                   object
RouteLetter                      object
RouteNumber                     float64
RouteDescription                 object
HectometerDirection              object
HeadSegmentFrom                  object
HeadSegmentTo                    object
TrajectoryFrom                   object
TrajectoryTo                     object
CauseGroundDetail                object
CauseProgress                    object
CauseCodeProgress                object
CauseCode                        object
Cause1                           object
Cause2                           object
Cause3                           object


In [85]:
import pandas as pd

# Assuming merged_data is already loaded and processed up to the previous steps

# Transform 'YYYYMMDD' in 'Date' to 'dd/mm/yyyy' format
merged_data['Date'] = pd.to_datetime(merged_data['Date'], format='%Y%m%d').dt.strftime('%d/%m/%Y')

# Replace NaN values in 'DateFileStart' and 'DateFileEnd' with 'Date'
merged_data['DateFileStart'] = merged_data['DateFileStart'].fillna(pd.to_datetime(merged_data['Date'], format='%d/%m/%Y'))
merged_data['DateFileEnd'] = merged_data['DateFileEnd'].fillna(pd.to_datetime(merged_data['Date'], format='%d/%m/%Y'))

# Replace NaN values in 'TimeFileStart' and 'TimeFileEnd' with 'Hour' and extract only the time part
merged_data['TimeFileStart'] = merged_data['TimeFileStart'].fillna(pd.to_datetime(merged_data['Hour'], format='%H:%M:%S').dt.time)
merged_data['TimeFileEnd'] = merged_data['TimeFileEnd'].fillna(pd.to_datetime(merged_data['Hour'], format='%H:%M:%S').dt.time)

# Ensure 'TimeFileStart' and 'TimeFileEnd' are in the correct format
merged_data['TimeFileStart'] = merged_data['TimeFileStart'].apply(lambda x: x.strftime('%H:%M:%S') if not pd.isnull(x) else x)
merged_data['TimeFileEnd'] = merged_data['TimeFileEnd'].apply(lambda x: x.strftime('%H:%M:%S') if not pd.isnull(x) else x)

# Replace NaN values in specified columns with 0
columns_to_fill = ['NLSituationNumber', 'FileSeverity', 'AvgLength', 'FileDuration',
                   'HectometerHead', 'HectometerTail', 'RouteNumber', 'HectometerDirection', 'HeadSegmentFrom', 
                   'HeadSegmentTo', 'TrajectoryFrom', 'TrajectoryTo']

merged_data[columns_to_fill] = merged_data[columns_to_fill].fillna(0)

# Replace NaN values in specified columns with "No Incident"
merged_data['CauseGroundDetail'] = merged_data['CauseGroundDetail'].fillna("No Incident")

# Drop unnecessary columns
columns_to_drop = ['RouteLetter', 'RouteDescription', 'CauseProgress', 'CauseCodeProgress', 'Cause1', 'Cause2', 'Cause3', 'Cause4']
merged_data = merged_data.drop(columns=columns_to_drop)

merged_data

,NLSituationNumber,DateFileStart,DateFileEnd,TimeFileStart,TimeFileEnd,FileSeverity,AvgLength,FileDuration,HectometerHead,HectometerTail,...,Visibility,CloudCover,Humidity,Weather,WeatherCode,Fog,Rain,Snow,Thunder,IceFormation
0,0.0,2019-01-01,2019-01-01,01:00:00,01:00:00,0,0,0,0,0,...,31,8,97,10,7,0,0,0,0,0
1,0.0,2019-01-01,2019-01-01,02:00:00,02:00:00,0,0,0,0,0,...,25,8,96,10,7,0,0,0,0,0
2,0.0,2019-01-01,2019-01-01,03:00:00,03:00:00,0,0,0,0,0,...,29,8,94,10,7,0,0,0,0,0
3,0.0,2019-01-01,2019-01-01,04:00:00,04:00:00,0,0,0,0,0,...,50,8,94,10,7,0,0,0,0,0
4,0.0,2019-01-01,2019-01-01,05:00:00,05:00:00,0,0,0,0,0,...,58,8,94,10,7,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378984,0.0,2024-03-25,2024-03-25,20:00:00,20:00:00,0,0,0,0,0,...,75,8,73,,5,0,0,0,0,0
378985,0.0,2024-03-25,2024-03-25,21:00:00,21:00:00,0,0,0,0,0,...,75,8,79,,5,0,0,0,0,0
378986,0.0,2024-03-25,2024-03-25,22:00:00,22:00:00,0,0,0,0,0,...,75,8,73,,5,0,0,0,0,0
378987,0.0,2024-03-25,2024-03-25,23:00:00,23:00:00,0,0,0,0,0,...,75,8,77,,5,0,0,0,0,0


In [86]:
import datetime as dt

public_holidays = [
    '2019-01-01', '2019-04-19', '2019-04-21', '2019-04-22', '2019-04-27',
    '2019-05-05', '2019-05-30', '2019-06-09', '2019-06-10', '2019-12-25', '2019-12-26',
    '2020-01-01', '2020-04-10', '2020-04-12', '2020-04-13', '2020-04-27', '2020-05-05',
    '2020-05-21', '2020-05-31', '2020-06-01', '2020-12-25', '2020-12-26',
    '2021-01-01', '2021-04-02', '2021-04-04', '2021-04-05', '2021-04-27', '2021-05-05',
    '2021-05-13', '2021-05-23', '2021-05-24', '2021-12-25', '2021-12-26',
    '2022-01-01', '2022-04-15', '2022-04-17', '2022-04-18', '2022-04-27', '2022-05-05',
    '2022-05-26', '2022-06-05', '2022-06-06', '2022-12-25', '2022-12-26',
    '2023-01-01', '2023-04-07', '2023-04-09', '2023-04-10', '2023-04-27', '2023-05-05',
    '2023-05-18', '2023-05-28', '2023-05-29', '2023-12-25', '2023-12-26',
    '2024-01-01', '2024-03-29', '2024-03-31', '2024-04-01', '2024-04-27', '2024-05-05',
    '2024-05-09', '2024-05-19', '2024-05-20', '2024-12-25', '2024-12-26'
]

public_holidays = pd.to_datetime(public_holidays)

In [87]:
# Define working hours constraints
morning_start = dt.time(8, 30)      # 08:30 AM
morning_end = dt.time(9, 0)         # 09:00 AM
afternoon_start = dt.time(14, 0)    # 02:00 PM

# Minimum and Maximum day start and end
earliest_start = dt.time(7, 30)  # 07:30 AM
latest_end = dt.time(18, 30)     # 06:30 PM

- Records corresponding to weekends (Saturdays and Sundays) were removed. This was done by checking the day of the week for DateFileStart and excluding records where the day was either 5 (Saturday) or 6 (Sunday).
- A list of public holidays from 2019 to 2024 was created and converted to datetime format.
Records corresponding to these public holidays were removed by checking if DateFileStart matched any date in the public holidays list.

In [88]:
# Filter out weekends
merged_data = merged_data[~merged_data['DateFileStart'].dt.dayofweek.isin([5, 6])]

# Filter out public holidays
merged_data = merged_data[~merged_data['DateFileStart'].isin(public_holidays)]

# Reset the index
merged_data = merged_data.reset_index(drop=True)

merged_data

,NLSituationNumber,DateFileStart,DateFileEnd,TimeFileStart,TimeFileEnd,FileSeverity,AvgLength,FileDuration,HectometerHead,HectometerTail,...,Visibility,CloudCover,Humidity,Weather,WeatherCode,Fog,Rain,Snow,Thunder,IceFormation
0,0.0,2019-01-02,2019-01-02,01:00:00,01:00:00,0,0,0,0,0,...,67,8,70,,5,0,0,0,0,0
1,0.0,2019-01-02,2019-01-02,02:00:00,02:00:00,0,0,0,0,0,...,66,8,67,,5,0,0,0,0,0
2,0.0,2019-01-02,2019-01-02,03:00:00,03:00:00,0,0,0,0,0,...,64,8,71,,5,0,0,0,0,0
3,0.0,2019-01-02,2019-01-02,04:00:00,04:00:00,0,0,0,0,0,...,64,3,75,22,7,0,1,0,0,0
4,0.0,2019-01-02,2019-01-02,05:00:00,05:00:00,0,0,0,0,0,...,74,2,78,,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318132,0.0,2024-03-25,2024-03-25,20:00:00,20:00:00,0,0,0,0,0,...,75,8,73,,5,0,0,0,0,0
318133,0.0,2024-03-25,2024-03-25,21:00:00,21:00:00,0,0,0,0,0,...,75,8,79,,5,0,0,0,0,0
318134,0.0,2024-03-25,2024-03-25,22:00:00,22:00:00,0,0,0,0,0,...,75,8,73,,5,0,0,0,0,0
318135,0.0,2024-03-25,2024-03-25,23:00:00,23:00:00,0,0,0,0,0,...,75,8,77,,5,0,0,0,0,0


In [89]:
import datetime as dt

def convert_to_time(time_value):
    if isinstance(time_value, dt.time):
        return time_value
    try:
        return dt.datetime.strptime(time_value, '%H:%M:%S').time()
    except (ValueError, TypeError):
        return None

# Apply the function to convert TimeFileStart and TimeFileEnd
merged_data['TimeFileStart'] = merged_data['TimeFileStart'].apply(convert_to_time)
merged_data['TimeFileEnd'] = merged_data['TimeFileEnd'].apply(convert_to_time)

print(merged_data[['TimeFileStart', 'TimeFileEnd']].head())

  TimeFileStart TimeFileEnd
0      01:00:00    01:00:00
1      02:00:00    02:00:00
2      03:00:00    03:00:00
3      04:00:00    04:00:00
4      05:00:00    05:00:00


In [90]:
# Apply the function to convert TimeFileStart and TimeFileEnd
merged_data['TimeFileStart'] = merged_data['TimeFileStart'].apply(convert_to_time)
merged_data['TimeFileEnd'] = merged_data['TimeFileEnd'].apply(convert_to_time)

# Filter records based on the specified working hours constraints and day start and end constraints
merged_data = merged_data[((merged_data['TimeFileStart'] >= morning_start) & (merged_data['TimeFileStart'] <= morning_end)) | 
                              (merged_data['TimeFileStart'] >= afternoon_start)]

merged_data = merged_data[(merged_data['TimeFileStart'] >= earliest_start) & (merged_data['TimeFileEnd'] <= latest_end)]

# Display the head of the combined dataset to verify the changes
merged_data

,NLSituationNumber,DateFileStart,DateFileEnd,TimeFileStart,TimeFileEnd,FileSeverity,AvgLength,FileDuration,HectometerHead,HectometerTail,...,Visibility,CloudCover,Humidity,Weather,WeatherCode,Fog,Rain,Snow,Thunder,IceFormation
8,0.0,2019-01-02,2019-01-02,09:00:00,09:00:00,0,0,0,0,0,...,65,8,83,23,7,0,1,0,0,0
13,0.0,2019-01-02,2019-01-02,14:00:00,14:00:00,0,0,0,0,0,...,75,4,61,,5,0,0,0,0,0
14,0.0,2019-01-02,2019-01-02,15:00:00,15:00:00,0,0,0,0,0,...,73,1,67,,5,0,0,0,0,0
15,0.0,2019-01-02,2019-01-02,16:00:00,16:00:00,0,0,0,0,0,...,71,3,72,,5,0,0,0,0,0
16,0.0,2019-01-02,2019-01-02,17:00:00,17:00:00,0,0,0,0,0,...,69,7,78,,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318126,0.0,2024-03-25,2024-03-25,14:00:00,14:00:00,0,0,0,0,0,...,76,0,49,,5,0,0,0,0,0
318127,0.0,2024-03-25,2024-03-25,15:00:00,15:00:00,0,0,0,0,0,...,80,0,44,,5,0,0,0,0,0
318128,0.0,2024-03-25,2024-03-25,16:00:00,16:00:00,0,0,0,0,0,...,81,1,46,,5,0,0,0,0,0
318129,0.0,2024-03-25,2024-03-25,17:00:00,17:00:00,0,0,0,0,0,...,81,3,53,,5,0,0,0,0,0


`Earliest Start Time:`:
- The condition (df_copy['TimeFileStart'] >= earliest_start) ensures that no records start before 7:30 AM.

`Latest End Time`:
- The condition (df_copy['TimeFileEnd'] <= latest_end) ensures that no records end after 6:30 PM.

`Combining Conditions`:
- The two conditions are combined using the OR operator |. This ensures that a record is included if it satisfies either the morning window condition or the afternoon window condition.
- Start and end time conditions are applied in conjunction with the previously defined working hours constraints to ensure all constraints are respected.

In [95]:
# Define categories and their corresponding incidents
incident_categories = {
    "Accidents": [
        "Ongeval(len)", "Ongeval (met opruim/berging)", "Ongeval met vrachtwagen(s)",
        "Ongeval op aansluitende weg", "Ongeval (en weersomstandigheden)", "Ongeval (in een spitsfile)",
        "Ongeval vrachtwagen (met opruim/berging)", "Ongeval op aansluitende weg (met spitsfile)",
        "Ongeval op aansluitende weg (en incident op aansluitende weg)", "Ongeval (met spoedreparatie)",
        "Ongeval (met schade RWS)", "Ongeval (met ongeval op aanslutiende weg)", "Ongeval vrachtwagen (in een spitsfile)",
        "Ongeval op stedelijke weg", "Ongeval (met ongeval en ongeval op aansluitende weg)", "Ongeval (met ander incident)",
        "Ongeval (met file op omleidingsroute)", "Ongeval (met object op de weg)", "Ongeval vrachtwagen (met spoedreparatie)",
        "Ongeval op provinciale weg", "Ongeval (met opruim/berging) (en defecte vrachtwagen)", "Ongeval vrachtwagen (met defecte vrachtwagen)",
        "Ongeval (met incident op aanslutiende weg)", "Ongeval (met opruim / berging en ongeval op aansluitende weg)",
        "Ongeval (en daarna defect voertuig)", "Incident (ongeval vrachtwagen) en incident (defect voertuig)",
        "Incident op aansluitende weg", "Incident op provinciale weg", "Incident op stedelijke weg", "Incident op andere rijrichting",
        "Spitsfile (met een incident op aansluitende weg)", "Incident op aansluitende weg en spitsfile", "Spitsfile (met incident op aansluitende weg)",
        "Eerder(e) ongeval(len)", "Eerder ongeval op aansluitende weg", "Eerder incident op aansluitende weg",
        "Afremmend verkeer als gevolg van kijkers naar ongeval(len)", "Spitsfile (met ongeval)", "Spitsfile (met ongeval op aansluitende weg)",
        "Spitsfile (met incident)", "Spitsfile (met een incident op aansluitende weg)", "Spitsfile (met ongeval vrachtwagen)", "Ongevalen(filebuitenspitszonderoorzaak)",
        "Spitsfile(metongevalvrachtauto)", "Ongeval(endaarnawerkzaamheden)", "Spitsfile(metongevalenongevalopaansluitendeweg)"
    ],
    "Traffic Jams": [
        "Spitsfile (geen oorzaak gemeld)",
        "File buiten spits (geen oorzaak gemeld)","Spitsfile (met vrachtwagen)",
        "Spitsfile (geen oorzaak gemeld) met gevonden werk in Spin", "Extra verkeersdrukte door verkeer dat een omleiding volgt",
        "File op omleidingsroute (en file zonder oorzaak)", "Evenementen (en spitsfile)",
        "Evenementen (en files buiten spits zonder oorzaak)", "Spitsfile"
    ],
    "Vehicle Issues": [
        "Defect(e) voertuig(en)", "Defecte vrachtwagen(s)", "Defect voertuig (met opruim/berging)", "Defecte bus(sen)",
        "Defecte vrachtwagen (met opruim/berging)", "Defect voertuig (met opruim/berging en spitsfile)", "Defecte vrachtwagen (met ongeval)",
        "Defecte vrachtwagen (in een spitsfile)", "Defect voertuig (object op de weg)", "Defecte vrachtwagen (object op de weg in een spitsfile)",
        "Defect voertuig (en daarna ongeval)", "Defecte vrachtwagen (en ongeval)", "Defect voertuig (met spoedreparatie)",
        "Defecte vrachtwagen (met ongeval en opruim/berging)", "Defecte vrachtwagen (en files buiten spits zonder oorzaak)","Defecte vrachtwagen (object op de weg)",
        "Eerder pechgeval", "Spitsfile (met defect voertuig)", "Spitsfile (defect voertuig en berging)", "Teho(o)g(e)voertuig(en)", "Brandend(e)voertuig(en)",
        "Defectvoertuig(metspitsfile)", "Defectvoertuig(enanderincident)", "Defectevrachtwagen(enobjectopdewegineenspitsfile)", "Vehicle Issues"
    ],
    "Obstacles": [
        "Voorwerpen op de weg", "Object op de weg (met opruim/berging)", "Er ligt rommel op de weg", "Afgevallen lading(en)",
        "Geschaarde aanhanger(s)", "Geschaarde trekker(s) met oplegger", "Gekanteld(e) voertuig(en)", "Gekanteld(e) vrachtwagen(s)",
        "Mensen op de weg", "Dieren op de weg", "Omgewaaide bo(o)m(en)",
        "Gladde weg", "Water op de weg", "Wegdek in slechte toestand", "Modder op het wegdek", "Olie op het wegdek",
        "Zand op het wegdek", "Sneeuwval", "Winterse buien", "Onweersbuien", "Mist", "Spitsfile (met object op de weg)",
        "Beschadigdegeleiderail", "Objectopdeweg(ineenspitsfile)"
    ],
    "Roadworks and Maintenance": [
        "Spoedreparatie", "Wegwerkzaamheden", "Werkzaamheden op stedelijke weg", "Werkzaamheden op aansluitende weg",
        "Werkzaamheden in de berm", "Spitsstrook aan de rechterzijde gesloten", "Spitsstrook aan de linkerzijde gesloten",
        "Spitsstrook aan de rechterzijde versperd", "Spitsstrook aan de linkerzijde versperd", "Schade aan tunnel", "Schade aan wegmeubilair",
        "Verzakkingen", "Er wordt zout gestrooid", "Opgehoogde werkzaamheden", "Werkzaamheden (en daarna ongeval)",
        "Werkzaamheden (en file buiten spits zonder oorzaak)", "Werkzaamheden (in een spitsfile)", "Opruimwerkzaamheden",
        "Opruim/bergingswerkzaamheden (in een spitsfile)", "Opruimingswerkzaamheden",  "Spitsfile (en spoedreparatie)", "Spitsfile (en spoedreparatie)",
        "Extra verkeersdrukte door werkzaamheden elders", "Spitsfile(meteenincident)", "spoedreparatie(enspitsfile)", "Filebuitenspitszonderoorzaak(metwerkzaamheden)",
        "Roadworks and Maintenance" 
    ],
    "Emergency and Police": [
        "Betoging", "Politieonderzoek", "Politieactie", "Spoedtransport", "Ongevalsonderzoek"
    ],
    'Weather': [
        'Bermbrand', 'Sneeuwval', 'Onweersbuien', 'Winterse buien', 'Regenval', 'Mist', 'Falen DVM (en spitsfile)', 'Technische storing'
    ],
    "Other": [
        "Incident (met opruim/berging)", "Spitsfile (en schade RWS)", "Langza(a)m(e) voertuig(en)", "Spookrijder(s)",
        "Technische storing", "Verkeerslichten buiten werking", "Falen DVM (en spitsfile)", "Schade aan tunnel",
        "Schade aan wegmeubilair", "Verzakkingen", "Spitsfile (en schade RWS)", "Onwelwording", "Ophoogscript",
        "Brandindebuurtvandeweg", "Recreatie-envakantieverkeer", "schadeRWS(enspitsfile)", "Tehoogvoertuiggesignaleerd,wordtafgehandeld",
        "Tunnel gesloten", "Rijstrook voor hulpdiensten afgesloten", "Spitsstrook aan de rechterzijde gesloten",
        "Spitsstrook aan de linkerzijde gesloten", "Spitsstrook aan de rechterzijde versperd", "Spitsstrook aan de linkerzijde versperd"
    ]
}

# Reverse the mapping to have cause as key and category as value
reverse_cause_mapping = {cause.strip().replace(' ', ''): category for category, causes in incident_categories.items() for cause in causes}

# Strip any leading/trailing spaces and remove all spaces from the CauseGroundDetail values
merged_data['CauseGroundDetail'] = merged_data['CauseGroundDetail'].str.strip().str.replace(' ', '', regex=False)

# Replace old names with new categories
merged_data['CauseGroundDetail'] = merged_data['CauseGroundDetail'].replace(reverse_cause_mapping)

# Display the updated DataFrame
merged_data['CauseGroundDetail']


8         NoIncident
13        NoIncident
14        NoIncident
15        NoIncident
16        NoIncident
             ...    
318126    NoIncident
318127    NoIncident
318128    NoIncident
318129    NoIncident
318130    NoIncident
Name: CauseGroundDetail, Length: 151494, dtype: object

In [96]:
category_counts = merged_data['CauseGroundDetail'].value_counts()
print(category_counts)

CauseGroundDetail
TrafficJams                  128286
NoIncident                    18133
Accidents                      2977
Vehicle Issues                 1401
Roadworks and Maintenance       329
Obstacles                       172
Other                           135
Weather                          31
EmergencyandPolice               30
Name: count, dtype: int64


In [97]:
merged_data.to_csv('finally_Cline.csv', index=False)

In [98]:
merged_data

,NLSituationNumber,DateFileStart,DateFileEnd,TimeFileStart,TimeFileEnd,FileSeverity,AvgLength,FileDuration,HectometerHead,HectometerTail,...,Visibility,CloudCover,Humidity,Weather,WeatherCode,Fog,Rain,Snow,Thunder,IceFormation
8,0.0,2019-01-02,2019-01-02,09:00:00,09:00:00,0,0,0,0,0,...,65,8,83,23,7,0,1,0,0,0
13,0.0,2019-01-02,2019-01-02,14:00:00,14:00:00,0,0,0,0,0,...,75,4,61,,5,0,0,0,0,0
14,0.0,2019-01-02,2019-01-02,15:00:00,15:00:00,0,0,0,0,0,...,73,1,67,,5,0,0,0,0,0
15,0.0,2019-01-02,2019-01-02,16:00:00,16:00:00,0,0,0,0,0,...,71,3,72,,5,0,0,0,0,0
16,0.0,2019-01-02,2019-01-02,17:00:00,17:00:00,0,0,0,0,0,...,69,7,78,,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318126,0.0,2024-03-25,2024-03-25,14:00:00,14:00:00,0,0,0,0,0,...,76,0,49,,5,0,0,0,0,0
318127,0.0,2024-03-25,2024-03-25,15:00:00,15:00:00,0,0,0,0,0,...,80,0,44,,5,0,0,0,0,0
318128,0.0,2024-03-25,2024-03-25,16:00:00,16:00:00,0,0,0,0,0,...,81,1,46,,5,0,0,0,0,0
318129,0.0,2024-03-25,2024-03-25,17:00:00,17:00:00,0,0,0,0,0,...,81,3,53,,5,0,0,0,0,0
